# Bostrom Revenue and Price Model

In [1]:
from IPython.display import display, Markdown
from typing import Optional

from cyber_sdk.client.lcd import LCDClient
from cyber_sdk.client.lcd.params import PaginationOptions

from src.data_extractors import get_pools, get_prices
from config import BOSTROM_LCD_CLIENT, PUSSY_LCD_CLIENT, COSMOSHUB_LCD_CLIENT, CRESCENT_LCD_CLIENT, OSMOSIS_LCD_CLIENT

BASE_INVESTMINTING_AMOUNT_AMPERE = 100_000_000
BASE_INVESTMINTING_AMOUNT_VOLT = 1_000_000_000
BASE_INVESTMINTING_PERIOD_AMPERE = 30
BASE_INVESTMINTING_PERIOD_VOLT = 30

VALIDATOR_COMMISSION = 0.05
NUMBER_DEGEN_ITERATIONS = 3

## Optimal State
PRICE_LIQUID_OPTIMAL = 0.5
REVENUE_LIQUID_OPTIMAL = 0.4
BASE_REVENUE_OPTIMAL = 0.2  # stake + restake

## Actual State

In [2]:
def get_base_revenue(
        base_denom: str = 'boot', 
        liquid_denom: str = 'hydrogen',
        lcd_client: LCDClient = BOSTROM_LCD_CLIENT, 
        validator_commission: float = VALIDATOR_COMMISSION):
    
    _staked_base = lcd_client.staking.pool().bonded_tokens.amount
    _supply_base = \
        [int(_coin['amount']) 
         for _coin in lcd_client.bank.total(params=PaginationOptions(limit=10_000))[0].to_data() 
         if _coin['denom'] == base_denom][0]
    _inflation = float(lcd_client.mint.inflation())
    _community_tax = float(lcd_client.distribution.parameters()['community_tax'])

    _revenue_base = _supply_base / _staked_base * _inflation * (1 - validator_commission) * (1 - _community_tax)
    _revenue_with_restake_base = (1 + _revenue_base / 365) ** 365 - 1
    if base_denom not in ('boot', 'pussy'):
        return _staked_base, _supply_base, _revenue_base, _revenue_with_restake_base, None, None, None, None, _inflation, _community_tax
    _networks = ['bostrom', 'osmosis'] if base_denom == 'boot' else ['space-pussy']
    _price_df = get_prices(get_pools(networks=_networks), zero_fee=True)

    _price_ampere_actual = _price_df.loc[liquid_denom, 'milliampere' if base_denom == 'boot' else 'milliampere(pussy)'] * 1_000
    _price_volt_actual = _price_df.loc[liquid_denom, 'millivolt' if base_denom == 'boot' else 'millivolt(pussy)'] * 1_000
    _price_liquid_actual = _price_df.loc[base_denom, liquid_denom]
    return _staked_base, _supply_base, _revenue_base, _revenue_with_restake_base, _price_df, _price_ampere_actual, _price_volt_actual, _price_liquid_actual, _inflation, _community_tax

## Revenue Strategies
### Degen
Stake BOOT with restake -> Buy BOOT for HYDROGEN -> Stake BOOT with restake - 3 times

In [3]:
def get_degen_revenue(
        revenue_with_restake_base: float,
        price_liquid_actual: float,
        base_revenue_optimal: float = BASE_REVENUE_OPTIMAL,
        price_liquid_optimal: float = PRICE_LIQUID_OPTIMAL,
        number_degen_iterations: int = NUMBER_DEGEN_ITERATIONS
) -> [float, float, float, float]:
    
    _degen_revenue_actual = _degen_revenue_optimal = 0
    _staked_base_actual = _staked_base_optimal = 1

    for _ in range(number_degen_iterations):
        _degen_revenue_actual += _staked_base_actual * revenue_with_restake_base
        _staked_base_actual *= price_liquid_actual

        _degen_revenue_optimal += _staked_base_optimal * base_revenue_optimal
        _staked_base_optimal *= price_liquid_optimal
    return _degen_revenue_actual, _degen_revenue_optimal, _staked_base_actual, _staked_base_optimal

### Staking + Investminting

Stake BOOT with retake -> Investmint HYDROGEN

In [4]:
def get_staking_plus_investminting_revenue(
        price_volt_actual: float,
        price_ampere_actual: float,
        revenue_with_restake_base: float,
        price_liquid_actual: float,
        base_investminting_amount_volt: int = BASE_INVESTMINTING_AMOUNT_VOLT,
        base_investminting_period_volt: int = BASE_INVESTMINTING_PERIOD_VOLT,
        base_investminting_amount_ampere: int = BASE_INVESTMINTING_AMOUNT_AMPERE,
        base_investminting_period_ampere: int = BASE_INVESTMINTING_PERIOD_AMPERE,
        base_revenue_optimal: float = BASE_REVENUE_OPTIMAL,
        price_liquid_optimal: float = PRICE_LIQUID_OPTIMAL,
        revenue_liquid_optimal: float = REVENUE_LIQUID_OPTIMAL
) -> [float, float, float]:
    
    _liquid_revenue_actual = \
        (1 * 365 / (base_investminting_amount_volt * base_investminting_period_volt) * price_volt_actual +
         1 * 365 / (base_investminting_amount_ampere * base_investminting_period_ampere) * price_ampere_actual) / 2
    _staking_investminting_revenue_actual = revenue_with_restake_base + price_liquid_actual * _liquid_revenue_actual
    _staking_investminting_revenue_optimal = base_revenue_optimal + price_liquid_optimal * revenue_liquid_optimal
    return _liquid_revenue_actual, _staking_investminting_revenue_actual, _staking_investminting_revenue_optimal

## Price Model

In [5]:
def price_optimal(
        base_investminting_amount_volt: int = BASE_INVESTMINTING_AMOUNT_VOLT,
        base_investminting_period_volt: int = BASE_INVESTMINTING_PERIOD_VOLT,
        base_investminting_amount_ampere: int = BASE_INVESTMINTING_AMOUNT_AMPERE,
        base_investminting_period_ampere: int = BASE_INVESTMINTING_PERIOD_AMPERE,
        revenue_liquid_optimal: float = REVENUE_LIQUID_OPTIMAL
) -> [float, float]:
    
    _price_ampere_optimal = base_investminting_amount_ampere * base_investminting_period_ampere / 365 * revenue_liquid_optimal
    _price_volt_optimal = base_investminting_amount_volt * base_investminting_period_volt / 365 * revenue_liquid_optimal
    return _price_ampere_optimal, _price_volt_optimal

## Display data

In [6]:
def display_data(
        chain_name: str = 'Bostrom',
        base_denom: str = 'boot', 
        liquid_denom: Optional[str] = 'hydrogen',  
        liquid_denom_sn: Optional[str] = 'H', 
        lcd_client=BOSTROM_LCD_CLIENT) -> None:
    """
    Calculate, display the revenue and price model
    :param chain_name: chain name
    :param base_denom: chain base denom
    :param liquid_denom: liquid staking denom
    :param liquid_denom_sn: short name of liquid staking denom
    :param lcd_client: LCD client (Rest API) path
    :return: none
    """
    
    base_denom_uc = base_denom.upper()
    
    staked_base, supply_base, revenue_base, revenue_with_restake_base, price_df, price_ampere_actual, price_volt_actual, price_liquid_actual, inflation, community_tax = \
        get_base_revenue(base_denom=base_denom, liquid_denom=liquid_denom, lcd_client=lcd_client)
    
    display(Markdown(f'''
# {chain_name} Revenue and Price Model
## Actual state'''))
    print(f'{base_denom_uc} Supply {supply_base:>31,}\n'
          f'{base_denom_uc} Staked {staked_base:>31,}\n'
          f'{base_denom_uc} Inflation  {inflation:28.2%}\n'
          f'{base_denom_uc} Inflation Community Tax  {community_tax:14.2%}\n'
          f'{base_denom_uc} Staking Revenue without restake  {revenue_base:3.2%}\n'
          f'{base_denom_uc} Staking Revenue with restake     {revenue_with_restake_base:3.2%}\n')

    if chain_name not in ('Bostrom', 'Space-Pussy'):
        return 

    liquid_denom_uc = liquid_denom.upper()
    degen_revenue_actual, degen_revenue_optimal, staked_base_actual, staked_base_optimal = \
        get_degen_revenue(
            revenue_with_restake_base=revenue_with_restake_base,
            price_liquid_actual=price_liquid_actual)
    
    liquid_revenue_actual, staking_investminting_revenue_actual, staking_investminting_revenue_optimal = \
        get_staking_plus_investminting_revenue(
            price_volt_actual=price_volt_actual,
            price_ampere_actual=price_ampere_actual,
            revenue_with_restake_base=revenue_with_restake_base,
            price_liquid_actual=price_liquid_actual)
    
    price_ampere_optimal, price_volt_optimal = price_optimal()
    
    display(Markdown(f'''
## Revenue Strategies
### Degen
Stake {base_denom_uc} with restake -> Buy {base_denom_uc} for {liquid_denom_uc} -> Stake {base_denom_uc} with restake - 3 times'''))
    print(f'Degen revenue actual {degen_revenue_actual:7.1%}\n'
          f'Degen revenue optimal {degen_revenue_optimal:6.1%}')
    
    display(Markdown(f'''
### Staking + Investminting
Stake {base_denom_uc} with retake -> Investmint {liquid_denom_uc}'''))
    print(f'Staking + Investminting revenue actual {staking_investminting_revenue_actual:7.1%}\n'
          f'Staking + Investminting revenue optimal {staking_investminting_revenue_optimal:6.1%}')
    
    display(Markdown(f'''
### Investminting
Buy {liquid_denom_uc} for {base_denom_uc} -> Investmint {liquid_denom_uc}'''))
    print(f'Investminting revenue actual {liquid_revenue_actual:7.1%}\n'
          f'Investminting revenue optimal {REVENUE_LIQUID_OPTIMAL:6.1%}')
    
    display(Markdown('''## Price Model'''))
    print(f'AMPERE price actual {price_ampere_actual:>,.0f} {liquid_denom_sn} optimal {price_ampere_optimal:>,.0f}, '
          f'AMPERE can decrease by {price_ampere_actual / price_ampere_optimal:.2f} times\n'
          f'VOLT price actual {price_volt_actual:>,.0f} {liquid_denom_sn} optimal {price_volt_optimal:>,.0f}, '
          f'VOLT can decrease by {price_volt_actual / price_volt_optimal:.2f} times\n'
          f'{liquid_denom_sn} price actual {price_liquid_actual:.3f} {base_denom_uc} optimal {PRICE_LIQUID_OPTIMAL}, '
          f'{liquid_denom_sn} can increase by {PRICE_LIQUID_OPTIMAL / price_liquid_actual :.2f} times\n')
    
    if chain_name == 'Bostrom':
        print(f"OSMO price"
              f"\n\t- in bostrom {price_df.loc[liquid_denom, 'uosmo in bostrom'] * 1_000_000:>,.0f} {liquid_denom_sn}"
              f"\n\t- in osmosis {price_df.loc['boot in osmosis', 'uosmo'] * price_df.loc[liquid_denom, base_denom] * 1_000_000:>,.0f} {liquid_denom_sn}")
        print(f"ATOM price"
              f"\n\t- in bostrom {price_df.loc[liquid_denom, 'uatom in bostrom'] * 1_000_000:>,.0f} H"
              f"\n\t- in osmosis {price_df.loc['boot in osmosis', 'uatom in osmosis'] * price_df.loc[liquid_denom, base_denom] * 1_000_000:>,.0f} {liquid_denom_sn}")

In [7]:
display_data()


# Bostrom Revenue and Price Model
## Actual state

BOOT Supply           1,184,768,201,944,181
BOOT Staked             286,664,648,013,238
BOOT Inflation                         5.46%
BOOT Inflation Community Tax          10.00%
BOOT Staking Revenue without restake  19.29%
BOOT Staking Revenue with restake     21.27%



## Revenue Strategies
### Degen
Stake BOOT with restake -> Buy BOOT for HYDROGEN -> Stake BOOT with restake - 3 times

Degen revenue actual   28.5%
Degen revenue optimal  35.0%



### Staking + Investminting
Stake BOOT with retake -> Investmint HYDROGEN

Staking + Investminting revenue actual   44.1%
Staking + Investminting revenue optimal  40.0%



### Investminting
Buy HYDROGEN for BOOT -> Investmint HYDROGEN

Investminting revenue actual   84.9%
Investminting revenue optimal  40.0%


## Price Model

AMPERE price actual 6,533,868 H optimal 3,287,671, AMPERE can decrease by 1.99 times
VOLT price actual 74,245,128 H optimal 32,876,712, VOLT can decrease by 2.26 times
H price actual 0.268 BOOT optimal 0.5, H can increase by 1.86 times

OSMO price
	- in bostrom 469,934,513 H
	- in osmosis 468,040,202 H
ATOM price
	- in bostrom 2,754,836,219 H
	- in osmosis 2,776,956,106 H


In [8]:
display_data(
    chain_name='Space-Pussy',
    base_denom='pussy', 
    liquid_denom='liquidpussy', 
    liquid_denom_sn='LP', 
    lcd_client=PUSSY_LCD_CLIENT)


# Space-Pussy Revenue and Price Model
## Actual state

PUSSY Supply       1,034,727,628,440,134,303
PUSSY Staked          28,164,863,927,011,930
PUSSY Inflation                         3.00%
PUSSY Inflation Community Tax          20.00%
PUSSY Staking Revenue without restake  83.76%
PUSSY Staking Revenue with restake     130.87%



## Revenue Strategies
### Degen
Stake PUSSY with restake -> Buy PUSSY for LIQUIDPUSSY -> Stake PUSSY with restake - 3 times

Degen revenue actual  177.1%
Degen revenue optimal  35.0%



### Staking + Investminting
Stake PUSSY with retake -> Investmint LIQUIDPUSSY

Staking + Investminting revenue actual  148.0%
Staking + Investminting revenue optimal  40.0%



### Investminting
Buy LIQUIDPUSSY for PUSSY -> Investmint LIQUIDPUSSY

Investminting revenue actual   61.9%
Investminting revenue optimal  40.0%


## Price Model

AMPERE price actual 5,544,173 LP optimal 3,287,671, AMPERE can decrease by 1.69 times
VOLT price actual 46,336,313 LP optimal 32,876,712, VOLT can decrease by 1.41 times
LP price actual 0.277 PUSSY optimal 0.5, LP can increase by 1.81 times


In [9]:
display_data(
    chain_name='CosmosHub',
    base_denom='uatom', 
    liquid_denom=None, 
    liquid_denom_sn=None, 
    lcd_client=COSMOSHUB_LCD_CLIENT)


# CosmosHub Revenue and Price Model
## Actual state

UATOM Supply             386,064,139,463,675
UATOM Staked             252,598,088,245,551
UATOM Inflation                        10.00%
UATOM Inflation Community Tax          10.00%
UATOM Staking Revenue without restake  13.07%
UATOM Staking Revenue with restake     13.96%
